# Scanpy Example Workflow
This is a quick example of how to process single cell data using scanpy

## Download the raw data
The first step is to download the raw data from GEO. This is from a published paper and can be found at [GSE131059](https://www.ncbi.nlm.nih.gov/geo/query/acc.cgi?acc=GSE131059)

Since this is older 10X data processed with cellranger v2- we also will need decompress the data. 

In [ ]:
%%bash
wget 'https://ftp.ncbi.nlm.nih.gov/geo/samples/GSM3762nnn/GSM3762870/suppl/GSM3762870_Car1gfp_barcodes.tsv.gz' -P ./data/car1/
wget 'https://ftp.ncbi.nlm.nih.gov/geo/samples/GSM3762nnn/GSM3762870/suppl/GSM3762870_Car1gfp_genes.tsv.gz' -P ./data/car1/
wget 'https://ftp.ncbi.nlm.nih.gov/geo/samples/GSM3762nnn/GSM3762870/suppl/GSM3762870_Car1gfp_matrix.mtx.gz' -P ./data/car1/

wget 'https://ftp.ncbi.nlm.nih.gov/geo/samples/GSM3762nnn/GSM3762869/suppl/GSM3762869_Naivebm_barcodes.tsv.gz' -P ./data/naive/
wget 'https://ftp.ncbi.nlm.nih.gov/geo/samples/GSM3762nnn/GSM3762869/suppl/GSM3762869_Naivebm_genes.tsv.gz' -P ./data/naive/
wget 'https://ftp.ncbi.nlm.nih.gov/geo/samples/GSM3762nnn/GSM3762869/suppl/GSM3762869_Naivebm_matrix.mtx.gz' -P ./data/naive/

gzip -d ./data/car1/*.gz
gzip -d ./data/naive/*.gz

cd ./data/naive
ls * | while read fn; do
    mv "${fn}" "${fn/GSM3762869_Naivebm_/}";
done

cd ../car1/
ls * | while read fn; do
    mv "${fn}" "${fn/GSM3762870_Car1gfp_/}";
done

cd ../../

## Import libraries
Now we need to import the basic libraries we will be working with. These all should be included in the binder conda environment. 

In [ ]:
import numpy as np
import scanpy as sc
import pandas as pd
import bbknn
import anndata

## Set up a few functions
Next for ease, we can set up a few functions for processing the data. Since we would be using these repeatedly (and maybe one day for more samples than this project), its advised to set up functions. Use the minimal code you need to get the job done. 

In [ ]:
def LoadData(gex_path):
    #Load the 10X files as matrix files
    adata = sc.read_10x_mtx(gex_path, cache_compression = None, var_names='gene_symbols')
    #Make the gene names unique in case of overlap
    adata.var_names_make_unique()
    return (adata)

def BasicFiltering(adata):
    #Quickly show the top 20 highest expressed genes
    sc.pl.highest_expr_genes(adata, n_top=20,)

    #Show the quick cell counts pre/post filtering
    print("Before filtering:", adata.n_obs, adata.n_vars)
    sc.pp.filter_genes(adata, min_cells=3) # keep the genes which are expressed in min number of cells
    sc.pp.filter_cells(adata, min_genes=200) # cells with min number of genes expressed
    print("After filtering:", adata.n_obs, adata.n_vars)

    #Pull out the mitochondrial, ribosomal, and hemoglobin counts
    #These patterns will have to change per species. IE mouse/human/rat etc. 
    adata.var['mt'] = adata.var_names.str.startswith('mt-')  
    adata.var['ribo']=adata.var_names.str.startswith(('Rps','Rpl'))
    adata.var['hemo']=adata.var_names.str.startswith(('^Hb[^(b)]'))
    sc.pp.calculate_qc_metrics(adata, qc_vars=['mt','ribo','hemo'], percent_top=None, inplace=True, log1p=False)
    sc.pl.violin(adata, ['n_genes_by_counts', 'total_counts', 'pct_counts_mt','pct_counts_ribo','pct_counts_hemo'], jitter=0.4, multi_panel=True)

    #Remove cells with higher gene counts. These are putative doublets.
    adata = adata[adata.obs.n_genes_by_counts < 5000, :]
    print("Remaining cells after removing high number of genes %d"%adata.n_obs)

    #Likewise filter low counts. These are putative noise GEMs
    adata = adata[adata.obs.n_genes_by_counts > 500, :]
    print("Remaining cells after removing high number of genes %d"%adata.n_obs)

    #Filter based on mitochondrial counts. These are proxies for dead/dying cells. You can also filter for hemoglobin and ribosomal as well. 
    adata = adata[adata.obs.pct_counts_mt < 10, :] #10% mitochondrial contamination is a solid cutoff to start with. 
    print("Remaining cells afer removing mt %d"%adata.n_obs)

    #Perform the normalization
    sc.pp.normalize_per_cell(adata, counts_per_cell_after=1000)
    sc.pp.log1p(adata)

    #Identify highly variable genes
    sc.pp.highly_variable_genes(adata, min_mean=0.0125, max_mean=3, min_disp=0.5)
    
    #Two last steps real quick. First to regress out the variation caused by the mitochondrial, ribosomal, and hemoglobin counts
    sc.pp.regress_out(adata, ['total_counts','pct_counts_mt','pct_counts_ribo','pct_counts_hemo'])
    #And then scale the data appropriately. 
    sc.pp.scale(adata)
    
    return(adata)
    
def Clustering(adata):
    sc.pp.pca(adata, svd_solver="arpack")
    sc.pp.neighbors(adata, n_neighbors=10, n_pcs=40)
    sc.tl.umap(adata)
    
    # Standard Leiden clustering
    sc.tl.leiden(adata)
    sc.pl.umap(adata, color=['leiden'])
    return (adata)

## Load and process
Once we have the functions built above, now we need to actually process the data. First we load each object, then add metadata tags, then perform basic filtering. 

In [ ]:
naive=LoadData('./data/naive/')
naive.obs['sample']='Naive'
naive=BasicFiltering(naive)

car1=LoadData('./data/car1/')
car1.obs['sample']='Car1'
car1=BasicFiltering(car1)

## Cluster the data
Now with our filtered data, we can start to cluster it and see what populations are present. 

In [ ]:
naive=Clustering(naive)

In [ ]:
car1=Clustering(car1)

## Merge the datasets
Since this is a Car1 KO vs Naive experiment, we would naturally want to see the differences between the two groupings. We can either merge the datasets or integrate them. First we will merge to see how they look in terms of batch effects. 

In [ ]:
var_names=naive.var_names.intersection(car1.var_names)
naive=naive_treated[:,var_names]
car1=car1[:,var_names]

adata = naive.concatenate(car1)

adata= Clustering(adata)

#And an extra plot to show the batch differences
sc.pl.umap(adata, color=['sample','leiden'])

## Integrate
Now that we can see the batch effects, we can integrate the samples and go from there. 

In [ ]:
sc.external.pp.bbknn(adata, batch_key='sample')

sc.tl.umap(adata)
sc.tl.leiden(adata)
sc.pl.umap(adata, color=['sample', 'leiden'])